<a href="https://colab.research.google.com/github/Farbod-gsm99/NLP-Tasks-and-Tokenizers/blob/main/Training_WordPiece_Tokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install datasets

In [ ]:
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)
from datasets import load_dataset

dataset = load_dataset("wikitext", name="wikitext-2-raw-v1", split="train")


def get_training_corpus():
    for i in range(0, len(dataset), 1000):
        yield dataset[i : i + 1000]["text"]

In [ ]:
tokenizer = Tokenizer(models.WordPiece(unk_token="[UNK]"))

tokenizer.normalizer = normalizers.Sequence(
    [normalizers.NFD(), normalizers.Lowercase(), normalizers.StripAccents()]
)

In [ ]:
pre_tokenizer = pre_tokenizers.Sequence(
    [pre_tokenizers.WhitespaceSplit(), pre_tokenizers.Punctuation()]
)
pre_tokenizer.pre_tokenize_str("Let's test my pre-tokenizer.")

In [ ]:
special_tokens = ["[UNK]", "[PAD]", "[CLS]", "[SEP]", "[MASK]"]
trainer = trainers.WordPieceTrainer(vocab_size=25000, special_tokens=special_tokens)

In [ ]:
tokenizer.train_from_iterator(get_training_corpus(), trainer = trainer)

In [ ]:
encoding = tokenizer.encode("Let's test this tokenizer.")
print(encoding.tokens)

In [ ]:
cls_token_id = tokenizer.token_to_id("[CLS]")
sep_token_id = tokenizer.token_to_id("[SEP]")
print(cls_token_id, sep_token_id)

In [ ]:
tokenizer.post_processor = processors.TemplateProcessing(
    single=f"[CLS]:0 $A:0 [SEP]:0",
    pair=f"[CLS]:0 $A:0 [SEP]:0 $B:1 [SEP]:1",
    special_tokens=[("[CLS]", cls_token_id), ("[SEP]", sep_token_id)],
)

In [ ]:
encoding = tokenizer.encode("Let's test this tokenizer.")
print(encoding.tokens)

In [ ]:
encoding = tokenizer.encode("Let's test this tokenizer...", "on a pair of sentences.")
print(encoding.tokens)
print(encoding.type_ids)

In [ ]:
tokenizer.decoder = decoders.WordPiece(prefix="##")

In [ ]:
tokenizer.decode(encoding.ids)

In [ ]:
tokenizer.save('tokenizer.json')

In [ ]:
from huggingface_hub import notebook_login, login

# For notebooks:
notebook_login()

In [ ]:
from transformers import BertTokenizerFast

wrapped_tokenizer = BertTokenizerFast(tokenizer_object=tokenizer)

In [ ]:
wrapped_tokenizer.push_to_hub('WordPiece-Tokenizer')